# `torch.nn.Embedding`

> ‘我一直对Embedding有一层抽象的，模糊的认识’

参考[我最爱的b站up主的内容](https://www.bilibili.com/video/BV1wm4y187Cr/?spm_id_from=333.337.search-card.all.click&vd_source=32f9de072b771f1cd307ca15ecf84087)

## embedding的基础概念

`embedding`是将词向量中的词映射为固定长度的词向量的技术，可以将one_hot出来的高维度的稀疏的向量转化成低维的连续的向量

![直观显示词与词之间的关系](../assets/lecture-pic/Embedding1.png)

## 首先明白embedding的计算过程

- embedding module 的前向过程是一个索引(查表)的过程
    - 表的形式是一个matrix （也即 embedding.weight,learnabel parameters）
        - matrix.shape:(v,h)
            - v:vocabulary size
            - h:hidden dimension

    - 具体的索引的过程，是通过onehot+矩阵乘法的形式实现的
    - input.shape:(b,s)
        - b: batch size
        - s: seq len 
    - embedding(input)=>(b,s,h)
    - **这其中关键的问题就是(b,s)和(v,h)怎么变成了(b,s,h)**

In [1]:
import torch
import torch.nn as nn
embedding = nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 9]])
embedding(input)

tensor([[[-0.4682, -1.2143,  0.1752],
         [ 1.5085,  0.4936,  0.3845],
         [-1.1064,  1.0143,  0.4442],
         [ 0.6037,  0.6854,  0.3562]],

        [[-1.1064,  1.0143,  0.4442],
         [ 1.0134,  0.2836, -0.6358],
         [ 1.5085,  0.4936,  0.3845],
         [ 1.5759, -0.5384, -0.0649]]], grad_fn=<EmbeddingBackward0>)

## One-Hot 矩阵乘法

目前`one_hot`可以很方便地在`torch.nn.functional`中进行调用，对于一个[batchsize,seqlength]的tensor，one_hot向量可以十分方便的将其转化为[batchsize,seqlength,numclasses],此时，再与[numclasses,h]进行相乘，从而得到最终的[b,s,v]

## 参数padding_idx

这个参数的作用是指定某个位置的梯度不进行更新，但是为什么不进行更新，以及在哪个位置不进行更新我还没搞明白....

In [3]:
# example with padding_idx
embedding = nn.Embedding(10, 3, padding_idx=0)
input = torch.LongTensor([[0, 2, 0, 5]])
embedding(input)
# example of changing `pad` vector
padding_idx = 0
embedding = nn.Embedding(3, 3, padding_idx=padding_idx)
embedding.weight
with torch.no_grad():
     embedding.weight[padding_idx] = torch.ones(3)
embedding.weight

Parameter containing:
tensor([[ 1.0000,  1.0000,  1.0000],
        [-0.0073,  0.8613, -0.4185],
        [-0.1206, -0.8382,  0.4391]], requires_grad=True)

## 关于max_norm

这个参数用于设置输出和权重参数是否经过了正则化。